# Projeto 2 - Ciência dos Dados

Nome: Fernando Bichuette Assumpção

Nome: Hélio Zaia Franciscon

___
# Classificador automático Naive Bayers


In [1]:
import math
import pandas as pd



___
### Proposta:

Nossa proposta será classificar os tweets como relevantes positivos ou negativos, sendo os positivos falando bem da companhia Burguer King, popularmente conhecida como "bk".
Para isso usaremos plavras-chave que serão usadas para classificar os tweets, sendo que, de acordo ou não com a presença dessas palavras-chave em um tweet, ele poderá ser classificado como positivo (falando bem do Burguer King), ou negativo (reclamando ou criticando o Burguer King)

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Limpando o arquivo

In [2]:
import emoji_list
#importando os dados do excel pelo pandas
dados=pd.read_excel("bk.xlsx",'Teste')
import re 

#fazendo a limpesa do arquivo
def limpar_caracteres(tabela, titulo):
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t','@','?']
    i2 = ['?', '!']
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    for e in itens:
        a = a.str.replace(e,'')
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@','')
    a = a.str.replace('?','')
    a = a.str.replace(',','.')
        
    tabela[titulo] = a

    return tabela
titulo="Teste"
tabela=limpar_caracteres(dados,titulo)

def split_tweet(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@bk':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'palavras': lista}, ignore_index=True)
                
    return nova
nova=split_tweet(dados,titulo)

def tabela_palavras(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']
dados_palavras=tabela_palavras(nova)
nova.head()

,palavras
0,"[comi, um, bk, antes, de, sair, da, barra, e, ..."
1,"[queria, uma, versao, de, poesia, acustica, 2,..."
2,"[o, quanto, eu, amo, o, bk, nao, ta, escrito]"
3,"[nolargjr, chinelasemalca, os, sanduiches, do,..."
4,"[yarara, impune, !, ttps//youtube/gstt, 5, db,..."


In [3]:
#separando o database em palavras
dados_palavras.head()

0     comi
1       um
2       bk
3    antes
4       de
Name: palavras, dtype: object

In [4]:
#contando quantas vezes cada palavra aparece
palavras=dados_palavras.value_counts()
palavras.head()

bk     165
e      105
de      86
um      65
que     62
Name: palavras, dtype: int64

In [5]:
prob = dados_palavras.value_counts(True)

In [6]:
#probabilidade de aparecer a palavra "bk" ao se sortear uma plavra aleatoria do database
prob['bk']

0.05253104106972302

In [7]:
#checando se o valor está coerente
prob.sum()

0.9999999999999999

In [8]:
#definido uma tabela de comentarios relevantes
relevante=dados.loc[(dados['Relevante/Irrelevante'])==1]
relevante.head()

,Teste,Relevante/Irrelevante
0,comi um bk antes de sair da barra e agr to jan...,1
3,nolargjr chinelasemalca os sanduiches do vikin...,1
5,e que o bk ta fazendo lanche veggie com hambur...,1
6,o novo lanche do bk foi filmado aonde eu traba...,1
11,quero muito um bk de frango vou chorar,1


In [9]:
#definindo uma tabela de comentarios irrelevantes
irrelevante=dados.loc[(dados['Relevante/Irrelevante'])==0]
irrelevante.head()

,Teste,Relevante/Irrelevante
1,queria uma versao de poesia acustica 2 so co...,0
2,o quanto eu amo o bk nao ta escrito,0
4,yarara impune ! ttps//youtube/gstt 5 db 9 l-u,0
7,jairbolsonaro edub_bk joofelipem vai tomar no ...,0
8,babebiburra naosejavagaba bk mandou bjs,0


In [10]:
splited_relevante=split_tweet(relevante,titulo)
splited_irrelevante=split_tweet(irrelevante,titulo)
splited_relevante.head()

,palavras
0,"[comi, um, bk, antes, de, sair, da, barra, e, ..."
1,"[nolargjr, chinelasemalca, os, sanduiches, do,..."
2,"[e, que, o, bk, ta, fazendo, lanche, veggie, c..."
3,"[o, novo, lanche, do, bk, foi, filmado, aonde,..."
4,"[quero, muito, um, bk, de, frango, vou, chorar]"


In [11]:
#retrando a coluna do index "Relevante/Irrelevante"
relevancia_relevante=relevante.iloc[0:,0:1]
relevancia_irrelevante=irrelevante.iloc[0:,0:1]
relevancia_relevante.head()


,Teste
0,comi um bk antes de sair da barra e agr to jan...
3,nolargjr chinelasemalca os sanduiches do vikin...
5,e que o bk ta fazendo lanche veggie com hambur...
6,o novo lanche do bk foi filmado aonde eu traba...
11,quero muito um bk de frango vou chorar


In [12]:
relevancia_irrelevante.head()

,Teste
1,queria uma versao de poesia acustica 2 so co...
2,o quanto eu amo o bk nao ta escrito
4,yarara impune ! ttps//youtube/gstt 5 db 9 l-u
7,jairbolsonaro edub_bk joofelipem vai tomar no ...
8,babebiburra naosejavagaba bk mandou bjs


In [13]:
#calculando quantas vezes cada palavra aparece nos tweets relevantes
var=split_tweet(relevancia_relevante,titulo)
splited_relevante = tabela_palavras(var)
prob_relevante = splited_relevante.value_counts(True)
num_palavras_relevante = splited_relevante.value_counts()
num_palavras_relevante.head()

bk     111
e       65
um      50
de      47
que     43
Name: palavras, dtype: int64

In [14]:
#calculando a probabilidade da ocorrencia de cada palavra no grupo dos tweets relevantes
prob_relevante.head()

bk     0.058054
e      0.033996
um     0.026151
de     0.024582
que    0.022490
Name: palavras, dtype: float64

In [15]:
#probabilidade de aparecer a palavra "bk" sorteado uma palavra aleatoria nos tweets relevantes
prob_relevante['bk']

0.05805439330543933

In [16]:
#checando se a soma das palavras corresponde a 100%
prob_relevante.sum()

0.9999999999999998

In [17]:
#Calculando a ocorrencia de cada palavra nos tweets irrelevantes
var=split_tweet(relevancia_irrelevante,titulo)
splited_irrelevante=tabela_palavras(var)
prob_irrelevante = splited_irrelevante.value_counts(True) 
num_palavras_irrelevante = splited_irrelevante.value_counts()
num_palavras_irrelevante.head()


bk    54
e     40
de    39
do    29
eu    21
Name: palavras, dtype: int64

In [18]:
#calculando a probabilidade de ocorrencia de cada palavra nos tweets irrelevantes
prob_irrelevante.head()

bk    0.043938
e     0.032547
de    0.031733
do    0.023596
eu    0.017087
Name: palavras, dtype: float64

In [19]:
#calculando a probabilidade de ser sorteada a palavra "bk" dentro dos tweets irrelevantes
prob_irrelevante["bk"]

0.043938161106590726

In [20]:
#checando se a soma corresponde a 100%
prob_irrelevante.sum()

1.0

In [21]:
#conferindo quantos tweets relevantes e irrelevantes temos no database
dados["Relevante/Irrelevante"].value_counts()

1    111
0     89
Name: Relevante/Irrelevante, dtype: int64

In [22]:
#calculando a quantidade total de palavras temos em nosso database
tamanho=[]
newnova=nova.iloc[0:,0]
for t in newnova:
    for b in t:
        tamanho.append(b)


In [23]:
#usamos o "nova" pois ele é a variavel que contem todos os tweets, sendo cada um deles uma lista
#onde as plavras compões os elementos, da seguinte forma:
nova.head()

,palavras
0,"[comi, um, bk, antes, de, sair, da, barra, e, ..."
1,"[queria, uma, versao, de, poesia, acustica, 2,..."
2,"[o, quanto, eu, amo, o, bk, nao, ta, escrito]"
3,"[nolargjr, chinelasemalca, os, sanduiches, do,..."
4,"[yarara, impune, !, ttps//youtube/gstt, 5, db,..."


In [24]:
#criando novas colunas para o database final
nova["Classificacao Manual"] = pd.Series(dados["Relevante/Irrelevante"])
nova["Classificacao_Naive"] = pd.Series()
nova["Resposta"] = pd.Series()

___
### Suavização de Laplace

In [25]:
#criando um contador para percorrer o database linha por linha
a=0
#criando as variaveis que serão usadas na suavização de Laplace
total=len(tamanho)
total_relevantes=len(num_palavras_relevante)
total_irrelevantes=len(num_palavras_irrelevante)
len_01=len(num_palavras_relevante)
len_00=len(num_palavras_irrelevante)
#fazendo a suavização de Laplace
for valor in nova["palavras"]:
    alpha = 1
    prob01 = 1
    prob00 = 1
    for val in valor:
        #caso a palavra não exista no nosso database, terá seu valor alterado, para uma melhor 
        #classificação do Naive Bayers
        if val not in num_palavras_relevante or val not in num_palavras_irrelevante:
            sec01 = alpha
            sec00 = alpha
        
        else:
            sec00=num_palavras_irrelevante[val] + alpha
            sec01=num_palavras_relevante[val] + alpha
            
            
        if val in num_palavras_relevante:
            prob01 *= (num_palavras_relevante[val] + alpha) / (total_relevantes + total*alpha)
        else: 
            prob01 *= sec01/(len(num_palavras_relevante)+total*alpha)
        
        if val in num_palavras_irrelevante:   
            prob00 *= (num_palavras_irrelevante[val] + alpha) / (total_irrelevantes + total*alpha)
        else:
            prob00 *= sec00/(len(num_palavras_irrelevante)+total*alpha)
       
    if prob01>prob00:
        prob_x = 1
        nova["Classificacao_Naive"][a] = prob_x
        if nova["Classificacao Manual"][a] == prob_x:
            nova["Resposta"][a] = "Positivo = True"
        elif nova["Classificacao Manual"][a] != prob_x:
            nova["Resposta"][a] = "Positivo = False"
            
    elif prob01<prob00:
        prob_x = 0
        nova["Classificacao_Naive"][a] = prob_x
        if nova["Classificacao Manual"][a] == prob_x:
            nova["Resposta"][a] ="Negativo = True"
        elif nova["Classificacao Manual"][a] != prob_x:
            nova["Resposta"][a] ="Negativo = False"
    a+=1
resultado=nova

C:\Users\helio\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helio\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helio\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\helio\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value i


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
Na tabela a seguir temos, respectivamente, os tweets (quebrados em listas),
a avaliação feita manualmente, a avaliação feita pelo Naive Byers e a comparação final, validando
a classificação feita pelo progama como, sendo:
___
#### Positivo = True:
Classificação correta de positivo, ou seja, tweet relevante
___
#### Negativo = True:
Classificação correta de negatico, ou seja, tweet irrelevante
___
#### Positivo = False:
Classificação errada de um tweet, ou seja, um tweet irrelevate foi tido como 
relevante pelo progama
___
#### Negativo = False:
Classificação errada de um tweet, ou seja, um tweet relevante foi tido como irrelevante pelo progama


In [26]:
resultado.head(10)

,palavras,Classificacao Manual,Classificacao_Naive,Resposta
0,"[comi, um, bk, antes, de, sair, da, barra, e, ...",1,1.0,Positivo = True
1,"[queria, uma, versao, de, poesia, acustica, 2,...",0,0.0,Negativo = True
2,"[o, quanto, eu, amo, o, bk, nao, ta, escrito]",0,1.0,Positivo = False
3,"[nolargjr, chinelasemalca, os, sanduiches, do,...",1,1.0,Positivo = True
4,"[yarara, impune, !, ttps//youtube/gstt, 5, db,...",0,0.0,Negativo = True
5,"[e, que, o, bk, ta, fazendo, lanche, veggie, c...",1,1.0,Positivo = True
6,"[o, novo, lanche, do, bk, foi, filmado, aonde,...",1,1.0,Positivo = True
7,"[jairbolsonaro, edub_bk, joofelipem, vai, toma...",0,0.0,Negativo = True
8,"[babebiburra, naosejavagaba, bk, mandou, bjs]",0,0.0,Negativo = True
9,"[td, dia, eu, ouvindo, bk, planos, djonga, lea...",0,0.0,Negativo = True


In [27]:
#checando a diferença entre as classificações feitas à mão e as definidas pelo Naive Byers
valid = pd.crosstab(resultado['Classificacao Manual'], resultado['Classificacao_Naive'])
valid.head()

Classificacao_Naive,0.0,1.0
Classificacao Manual,,
0,78,11
1,0,111


In [28]:
resp = resultado["Resposta"].value_counts(True)
print(resp*100)

Positivo = True     55.5
Negativo = True     39.0
Positivo = False     5.5
Name: Resposta, dtype: float64


In [29]:
#checando se a soma das probabilidades corresponde a 100%
resp.sum()

1.0

___
### Concluindo

Através da observação dos dados obtidos do classificador e de sua comparação com os dados classificados à mão, notamos que o progama obteve resultados notáveis, de forma que classificou apenas 5,5% dos tweets de maneira errada (classificando tweets irrelevantes como relevantes).

Essa alta taxa de sucesso pode ser devida ao numero limitado de tweets a serem avaliados, ou seja, se houvessem mais tweets para serem avaliados, a taxa de sucesso poderia ser alterada, e teriamos mais dados para consolidar definitivamente nosso classificador Naive Byers. A alta taxa de sucesso obtida no classificador pode ocorrer por algum erro de código, porém, se for esse o caso, ressalto que revisei o código inúmeras vezes e não encontrei tal erro.

A respeito da ausencia de falsos negativos, não há motivos explicitos para explicar tal fato, o que nos faz concluir que o classificador classificou corretamente a maioria dos tweets, de modo a não deixar falsos negativos e ao mesmo tempo, possuir uma baixa taxa (5,5%) de falsos positivos. Esse fato pode indicar uma pré-disposição do programa de classificar possíveis falsos negativos como falso positivo, porém não há certeza sobre esse fato, assim o toranado uma hipótese que poderia ser abordada para uma possível implementação do programa no futuro, com um maior database para testes 

Por fim concluimos que o progama de classificação Naive Bayers funciona, de forma precisa, possuindo uma alta taxa de classificações corretamente atribuidas aos tweets, Positivos Verdadeiros(55,5%),e Negativos Verdadeiros(39%), assim possuindo uma taxa de acerto em suas classificações de 94,5%.
